In [1]:
import pandas as pd
import numpy as np
import os
import math
from scipy import signal
import scipy.io
from scipy import signal
import mne
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dropout, BatchNormalization, Input,UpSampling1D
from tensorflow.keras.layers import concatenate, Lambda, Conv2D, MaxPooling2D, GlobalAveragePooling2D,LSTM,Activation
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle

In [ ]:


start = [0,50,100,150,200,250,300,350,400,450,500]
end  = [500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500]
tensorflow.keras.backend.clear_session()
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet

for i1 in range(1,2):
    for j1 in range(9,21):
        trainctr = 0
        st = start[i1]
        en = end[j1]
        diff = en-st
        sam = en-st;
        if diff<1000:
            sam = 1000
            X_train = np.zeros((1300,31,1000))
            Y_train = np.zeros((1300,))
        else:
            X_train = np.zeros((1300,31,diff))
            Y_train = np.zeros((1300,))
        print(X_train.shape)
        print(st,en,diff)
        if st==en:
            continue
        for fi in os.listdir('Segmented/'):
            eeg_sig = np.load(os.path.join('Segmented/',fi))
            eeg_sig_var = eeg_sig[st:en,:]
            firstdim = eeg_sig_var.shape[0]
            if firstdim<1000:
                eeg_sig_var = np.pad(eeg_sig_var,[(0, 1000-firstdim), (0, 0)], mode='constant', constant_values=0)
        
            _,char,trial = fi.split('_')
            trial,_ = trial.split('.')

            for i in range(0,31):
                X_train[trainctr,i,:] = np.transpose(eeg_sig_var[:,i])
            Y_train[trainctr] = ord(char)-65
            trainctr = trainctr+1
        
        X_train_final, X_test_final, Y_train_final, Y_test_final = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)
        for i in range(0,len(X_train_final)):
            X_train_final[i,:,:] = (X_train_final[i,:,:]-np.mean(X_train_final[i,:,:]))/np.std(X_train_final[i,:,:])

        for i in range(0,len(X_test_final)):
            X_test_final[i,:,:] = (X_test_final[i,:,:]-np.mean(X_test_final[i,:,:]))/np.std(X_test_final[i,:,:])
        
        model = EEGNet(nb_classes=26,Chans=31,Samples=sam)
#         print(model.summary())
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=20,restore_best_weights=True)
        model.fit(X_train_final, y=to_categorical(Y_train_final),validation_split=0.2,epochs=500, batch_size=8,verbose=1,callbacks=[es])
        pred = model.predict(X_test_final)
        Y_pred = np.argmax(pred,axis=1)
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        print(st,en,accuracy_score(Y_pred,Y_test_final)*100)
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")


(1300, 31, 1000)
50 950 900
Epoch 1/500
104/104 [==============================] - 13s 122ms/step - loss: 3.2503 - accuracy: 0.0445 - val_loss: 3.2232 - val_accuracy: 0.1058
Epoch 2/500
104/104 [==============================] - 13s 127ms/step - loss: 3.0310 - accuracy: 0.1779 - val_loss: 3.0839 - val_accuracy: 0.1538
Epoch 3/500
104/104 [==============================] - 13s 124ms/step - loss: 2.8836 - accuracy: 0.2272 - val_loss: 3.0126 - val_accuracy: 0.1346
Epoch 4/500
104/104 [==============================] - 13s 124ms/step - loss: 2.8006 - accuracy: 0.2656 - val_loss: 2.9479 - val_accuracy: 0.1298
Epoch 5/500
104/104 [==============================] - 13s 127ms/step - loss: 2.7268 - accuracy: 0.2728 - val_loss: 2.9081 - val_accuracy: 0.1587
Epoch 6/500
104/104 [==============================] - 14s 136ms/step - loss: 2.6794 - accuracy: 0.2969 - val_loss: 2.8834 - val_accuracy: 0.1538
Epoch 7/500
104/104 [==============================] - 13s 126ms/step - loss: 2.6403 - accuracy:

104/104 [==============================] - 13s 123ms/step - loss: 2.5080 - accuracy: 0.3257 - val_loss: 2.8725 - val_accuracy: 0.1490
Epoch 12/500
104/104 [==============================] - 13s 120ms/step - loss: 2.4999 - accuracy: 0.3233 - val_loss: 2.8805 - val_accuracy: 0.1779
Epoch 13/500
104/104 [==============================] - 13s 122ms/step - loss: 2.4886 - accuracy: 0.3534 - val_loss: 2.7818 - val_accuracy: 0.1923
Epoch 14/500
104/104 [==============================] - 13s 121ms/step - loss: 2.4540 - accuracy: 0.3245 - val_loss: 2.8015 - val_accuracy: 0.1731
Epoch 15/500
104/104 [==============================] - 13s 122ms/step - loss: 2.4333 - accuracy: 0.3570 - val_loss: 2.7979 - val_accuracy: 0.2067
Epoch 16/500
104/104 [==============================] - 13s 125ms/step - loss: 2.3895 - accuracy: 0.3738 - val_loss: 2.8105 - val_accuracy: 0.1779
Epoch 17/500
104/104 [==============================] - 13s 122ms/step - loss: 2.3580 - accuracy: 0.3762 - val_loss: 2.8315 - val_a